In this notebook we will process the synthetic Austen/food reviews data and convert it into feature vectors. In later notebooks these feature vectors will be the inputs to models which we will train and eventually use to identify spam. 

This notebook uses [term frequency-inverse document frequency](https://en.wikipedia.org/wiki/Tf–idf), or tf-idf, to generate feature vectors. Tf-idf is commonly used to summarise text data, and it aims to capture how important different words are within a set of documents. Tf-idf combines a normalized word count (or term frequency) with the inverse document frequency (or a measure of how common a word is across all documents) in order to identify words, or terms, which are 'interesting' or important within the document. 


We begin by loading in the data:

In [1]:
import pandas as pd
import os.path

df = pd.read_parquet(os.path.join("data", "training.parquet"))

To illustrate the computation of tf-idf vectors we will first implement the method on a sample of three of the documents we just loaded.   

In [2]:
import numpy as np

np.random.seed(0xc0ffeeee)
df_samp = df.sample(3)

In [3]:
pd.set_option('display.max_colwidth', -1) #ensures that all the text is visible
df_samp

<ipython-input-3-47bc71ff5b10>:1: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1) #ensures that all the text is visible


,index,label,text
26675,6675,spam,"Once I received the product and thought it was just like at the dog park or somewhere outside their home. I have tried many K-cup varieties. I had fresh herbs all winter long it was sitting on, apparently she would rather I just feed her the real thing. Please, share them with my co-workers a few weeks to go rancid, then reheated. She would tear the bag open as the treats can dry out, although I think I like a strong coffee, it was not clustered - sounds like your thing, you'll probably love these."
11130,11130,legitimate,"Elinor, while she waited in silence for the appearance of equal solicitude, on topics which had by nature the first claim on her. I do not believe Isabella has any fortune at all: but that will not signify to anyone here what he really is. It is hearty, but not at the Cottage, though that had been brought on by the entrance of a third to cheer a long evening."
33492,13492,spam,"I buy a new tea, thank you. This superior dog biscuit recipe contains only 7 primary ingredients and when it comes to buying this product again. The Babycook is so cute on all of the Happy Baby brand too ... also almost $2 a cup it is worth it! Have one more bag to get rid of my symptoms for all these conditions are relieved. This is one great product. Even brewing this at twice the price. I went home, did more research, saw that the good people can keep it half of it for several years."


We begin by computing the term frequency ('tf') of the words in the three texts above. We use the `token_pattern` parameter to specify that we only want to consider words (no numeric values). We limit the number of words (`max_features`) to 20, so that we can easily inspect the output. This means that only the 20 words which appear most frequently across the three texts will be represented. 

In [4]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(token_pattern='(?u)\\b[A-Za-z]\\w+\\b', max_features = 20)
counts = vectorizer.fit_transform(df_samp["text"])

In [5]:
vectorizer.get_feature_names() #shows all the words used as features for this vectorizer

['all',
 'at',
 'bag',
 'can',
 'for',
 'had',
 'is',
 'it',
 'like',
 'not',
 'of',
 'on',
 'product',
 'she',
 'that',
 'the',
 'these',
 'this',
 'to',
 'was']

In [6]:
counts

<3x20 sparse matrix of type '<class 'numpy.int64'>'
	with 43 stored elements in Compressed Sparse Row format>

In [7]:
print(counts.toarray()) 

[[1 1 1 1 0 1 0 3 3 1 0 1 1 2 0 5 1 0 1 3]
 [1 2 0 0 1 2 2 1 0 3 2 3 0 1 2 4 0 0 2 0]
 [2 1 1 1 2 0 3 5 0 0 3 1 2 0 1 4 1 4 2 0]]


Each row of the array corresponds to one of the texts, whilst the columns relate to the words considered in this vectorizer. (You can confirm that 'all' appears once in the first two texts, and twice in the third text, and so on.)

The next stage of the process is to use the results of the term frequency matrix to compute the tf-idf. 

The inverse document frequency (idf) for a particular word, or feature, is computed as (the log of) a ratio of the number of documents in a corpus to the number of documents which contain that feature (up to some constant factors). 

In [8]:
from sklearn.feature_extraction.text import TfidfTransformer

tfidf_transformer = TfidfTransformer()
df_tfidf = tfidf_transformer.fit_transform(counts)

In [9]:
print(df_tfidf.toarray())

[[0.09704499 0.09704499 0.12496309 0.12496309 0.         0.12496309
  0.         0.29113496 0.49293434 0.12496309 0.         0.09704499
  0.12496309 0.24992618 0.         0.48522494 0.12496309 0.
  0.09704499 0.49293434]
 [0.1119649  0.2239298  0.         0.         0.14417519 0.28835039
  0.28835039 0.1119649  0.         0.43252558 0.28835039 0.3358947
  0.         0.14417519 0.28835039 0.4478596  0.         0.
  0.2239298  0.        ]
 [0.16517551 0.08258776 0.10634677 0.10634677 0.21269355 0.
  0.31904032 0.41293878 0.         0.         0.31904032 0.08258776
  0.21269355 0.         0.10634677 0.33035103 0.10634677 0.55933291
  0.16517551 0.        ]]


Each row of the object above is the desired tf-idf vector for the relevant document. 

A major disadvantage of using a vectorizer is that it will be dependent upon the dictionary of words it sees when it is 'fit' to the data. As such, if we are presented with a new passage of text and wish to compute a feature vector for for that text we are required to know which word maps to which space of the vector. Keeping track of a dictionary is impractical and will lead to inefficiency. 

Furthermore, there are only "spaces" in the vectorizer for words that have been seen in the fitting stage. If a new text sample contains a word which was not present when the vectorizer was first fit, there will be no place in the feature vectors to count that word. 

With that in mind, we consider using a [hashing vectorizer](https://en.wikipedia.org/wiki/Feature_hashing). Words can be hashed to buckets, and the bucket count incremented. This will give us a counts matrix, like we saw above, which we can then compute the tf-idf matrix for, without the need to keep track of which column in the matrix any given word maps to. 

One disadvantage of this approach is that collisions will occur - with a finite set of buckets multiple words will hash to the same bucket. As such we are no longer computing an exact tf-idf matrix.

Furthermore we will not be able to recover the word (or words) associated with a bucket at a later time if we need them. (For our application this won't be needed.)

We fix the number of buckets at 2<sup>10</sup> = 1024, but you can try using a different number of buckets and see how the spam detection models are effected.  

In [10]:
from sklearn.feature_extraction.text import HashingVectorizer
BUCKETS=1024

hv = HashingVectorizer(norm=None, token_pattern='(?u)\\b[A-Za-z]\\w+\\b', n_features=BUCKETS, alternate_sign = False)
hv


HashingVectorizer(alternate_sign=False, n_features=1024, norm=None,
                  token_pattern='(?u)\\b[A-Za-z]\\w+\\b')

In [11]:
hvcounts = hv.fit_transform(df["text"])
hvcounts

<40000x1024 sparse matrix of type '<class 'numpy.float64'>'
	with 2489524 stored elements in Compressed Sparse Row format>

We can then go on to compute the "approximate" tf-idf matrix for this, by applying the tf-idf transformer to the hashed counts matrix.

In [12]:
tfidf_transformer = TfidfTransformer()
hvdf_tfidf = tfidf_transformer.fit_transform(hvcounts)

In [13]:
hvdf_tfidf

<40000x1024 sparse matrix of type '<class 'numpy.float64'>'
	with 2489524 stored elements in Compressed Sparse Row format>

These vectors have far too many dimensions for us to easily picture  them as points in space.  [Principal component analysis](https://en.wikipedia.org/wiki/Principal_component_analysis), or PCA, is a statistical technique that is over a century old; it takes observations in a high-dimensional space and maps them to a (potentially much) smaller number of dimensions. We'll see it in action now, using the [implementation from scikit-learn](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html#sklearn.decomposition.PCA).

(To learn a little more about PCA and an alternative technique, visit [the visualisation notebook](01-vectors-and-visualization.ipynb).)

In [14]:
#PCA projection so that output can be visualised

import sklearn.decomposition

DIMENSIONS = 2

pca2 = sklearn.decomposition.TruncatedSVD(DIMENSIONS)

pca_a = pca2.fit_transform(hvdf_tfidf)

In [15]:
pca_df = pd.DataFrame(pca_a, columns=["x", "y"])
pca_df.sample(10)

,x,y
9232,0.512568,0.145887
23870,0.413900,-0.117440
8795,0.492844,0.137295
27274,0.468245,-0.071868
28240,0.284845,-0.043177
27157,0.178468,-0.058348
34899,0.273773,-0.123227
39898,0.382309,-0.182119
21788,0.382722,-0.135509
13047,0.404117,0.236167


In [16]:
plot_data = pd.concat([df.reset_index(), pca_df], axis=1)

In [1]:
import altair as alt

alt.Chart(plot_data.sample(1000)).encode(x="x", y="y", color="label").mark_point().interactive()

NameError: name 'plot_data' is not defined

We want to be able to easily compute feature vectors using the hashing tf-idf workflow laid out above. The `Pipeline` facility in scikit-learn streamlines this workflow by making it easy to pass data through multiple transforms. In the next cell we set up our pipeline.

In [18]:
from sklearn.feature_extraction.text import HashingVectorizer,TfidfTransformer
from sklearn.pipeline import Pipeline
import pickle, os

vect = HashingVectorizer(norm=None, token_pattern='(?u)\\b[A-Za-z]\\w+\\b', n_features=BUCKETS, alternate_sign = False)
tfidf = TfidfTransformer()

feat_pipeline = Pipeline([
    ('vect',vect),
    ('tfidf',tfidf)
])

We can then use the `fit_transform` method to apply the pipeline to our data frame. This produces a sparse matrix (only non zero entries are recorded). We convert this to a dense array using the `toarray()` function, then append the index and labels to aid readability. 

In [19]:
feature_vecs = feat_pipeline.fit_transform(df["text"]).toarray()
labeled_vecs = pd.concat([df.reset_index()[["index", "label"]],
                                pd.DataFrame(feature_vecs)], axis=1)
labeled_vecs.columns = labeled_vecs.columns.astype(str)

In [20]:
labeled_vecs.sample(10)

,index,label,0,1,2,3,4,5,6,7,...,1014,1015,1016,1017,1018,1019,1020,1021,1022,1023
36894,16894,spam,0.0,0.182364,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
37926,17926,spam,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
15271,15271,legitimate,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.191168,...,0.0,0.0,0.000000,0.178111,0.0,0.0,0.0,0.0,0.0,0.0
9223,9223,legitimate,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
24360,4360,spam,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
20085,85,spam,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
25807,5807,spam,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.130752,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
36553,16553,spam,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
16260,16260,legitimate,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
9916,9916,legitimate,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.146217,...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0


We save the feature vectors to a parquet file.

In [21]:
labeled_vecs.to_parquet(os.path.join("data", "features.parquet"))

We will then serialize our pipeline to a file on disk so that we can reuse the document frequencies we've observed on training data to weight term vectors.

In [22]:
from mlworkflows import util
util.serialize_to(feat_pipeline, "feature_pipeline.sav")

Now that we have a feature engineering approach, next step is to train a model.  Again, you have two choices for your next step:  [click here](04-model-logistic-regression.ipynb) for a model based on *logistic regression*, or [click here](04-model-random-forest.ipynb) for a model based on *ensembles of decision trees.*